<a href="https://colab.research.google.com/github/dmcg89/tripletloss/blob/master/tripletloss_model_gray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys,os
root_path = 'gdrive/My Drive/Colab Notebooks/tripletloss/'
sys.path.append(root_path)

In [4]:
from preprocessing1 import PreProcessing
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
from keras.callbacks import ModelCheckpoint




Using TensorFlow backend.


In [5]:
dataset = PreProcessing(root_path + 'ExtendedYaleB/', 'negative')

yaleB36
Failed to read images from Directory:  yaleB36
Exception Message:  cannot identify image file 'gdrive/My Drive/Colab Notebooks/tripletloss/ExtendedYaleB/yaleB36/yaleB36_P01.info'
yaleB12
Failed to read images from Directory:  yaleB12
Exception Message:  cannot identify image file 'gdrive/My Drive/Colab Notebooks/tripletloss/ExtendedYaleB/yaleB12/yaleB12_P07.info'
yaleB23
Failed to read images from Directory:  yaleB23
Exception Message:  cannot identify image file 'gdrive/My Drive/Colab Notebooks/tripletloss/ExtendedYaleB/yaleB23/yaleB23_P05.info'
yaleB24
Failed to read images from Directory:  yaleB24
Exception Message:  cannot identify image file 'gdrive/My Drive/Colab Notebooks/tripletloss/ExtendedYaleB/yaleB24/yaleB24_P02.info'
yaleB38
Failed to read images from Directory:  yaleB38
Exception Message:  cannot identify image file 'gdrive/My Drive/Colab Notebooks/tripletloss/ExtendedYaleB/yaleB38/yaleB38_P03.info'
yaleB25
Failed to read images from Directory:  yaleB25
Exception 

In [0]:
anchors = []
positives = []
negatives = []
n = 1000

for _ in range(n):
    triplet = dataset.generate_triplets()
    anchors.append(triplet[0])
    positives.append(triplet[1])
    negatives.append(triplet[2])

In [7]:
import numpy as np
def train_test_split(arr, p=.9):
    test = []
    train = []
    for i in range(len(arr)):
        if i < len(arr)*.9: train.append(arr[i])
        else: test.append(arr[i])
    return np.array(train), np.array(test)
#     return train, test

anchor_train, anchor_test = train_test_split(anchors)
print((anchor_train[1].shape))
print((anchor_train[3].shape))
positive_train, positive_test = train_test_split(positives)
negative_train, negative_test = train_test_split(negatives)

(480, 640)
(480, 640)


In [0]:
def triplet_loss(y_true, y_pred, alpha = 0.4):
    """
    Implementation of the triplet loss function
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    Returns:
    loss -- real number, value of the loss
    """
    print('y_pred.shape = ',y_pred)
    
    total_lenght = y_pred.shape.as_list()[-1]
#     print('total_lenght=',  total_lenght)
#     total_lenght =12
    
    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]

    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)

    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss

In [0]:
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import os
from keras.callbacks import ModelCheckpoint
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

def create_base_network(in_dims):
    """
    Base network to be shared.
    """
    model = Sequential()
    model.add(Conv2D(128,(7,7),padding='same',input_shape=(in_dims[0],in_dims[1],in_dims[2],),activation='relu',name='conv1'))
    model.add(MaxPooling2D((2,2),(2,2),padding='same',name='pool1'))
    model.add(Conv2D(256,(5,5),padding='same',activation='relu',name='conv2'))
    model.add(MaxPooling2D((2,2),(2,2),padding='same',name='pool2'))
    model.add(Flatten(name='flatten'))
    model.add(Dense(4,name='embeddings'))
    # model.add(Dense(600))
    
    return model

In [0]:
adam_optim = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)

In [11]:
anchor_input = Input((480, 640, 1), name='anchor_input')
positive_input = Input((480, 640, 1), name='positive_input')
negative_input = Input((480, 640, 1), name='negative_input')

# Shared embedding layer for positive and negative items
Shared_DNN = create_base_network([480, 640, 1])


encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)


merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')

model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
model.compile(loss=triplet_loss, optimizer=adam_optim)






y_pred.shape =  Tensor("merged_layer/concat:0", shape=(?, 12), dtype=float32)


In [12]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 480, 640, 1)  0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 480, 640, 1)  0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 480, 640, 1)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4)            20486660    anchor_input[0][0]               
                                                                 positive_input[0][0]       

In [0]:
import numpy as np
# RGB Images
# Anchor_train = anchor_train.reshape(-1,368,368,3)
# Positive_train = positive_train.reshape(-1,368,368,3)
# Negative_train = negative_train.reshape(-1,368,368,3)
# Anchor_test = anchor_test.reshape(-1,368,368,3)
# Positive_test = positive_test.reshape(-1,368,368,3)
# Negative_test = negative_test.reshape(-1,368,368,3)
# (480, 640, 1)
# Greyscale Images
Anchor_train = anchor_train.reshape(-1,480, 640, 1)
Positive_train = positive_train.reshape(-1,480, 640, 1)
Negative_train = negative_train.reshape(-1,480, 640, 1)
Anchor_test = anchor_test.reshape(-1,480, 640, 1)
Positive_test = positive_test.reshape(-1,480, 640, 1)
Negative_test = negative_test.reshape(-1,480, 640, 1)

In [0]:
import numpy as np
Y_dummy = np.empty((Anchor_train.shape[0],12))
Y_dummy2 = np.empty((Anchor_test.shape[0],12))

n_batch = 10

checkpointer = ModelCheckpoint(filepath=root_path + "weights.hdf5", verbose=1, save_best_only=True)
# checkpointer = ModelCheckpoint(filepath="/tmp/weights.hdf5", verbose=1)
model.fit([Anchor_train,Positive_train,Negative_train],y=Y_dummy,validation_data=([Anchor_test,Positive_test,Negative_test],Y_dummy2),callbacks=[checkpointer], epochs=10, batch_size=n_batch)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 900 samples, validate on 100 samples
Epoch 1/10





900/900 [==============================] - 10021s 11s/step - loss: 0.0494 - val_loss: 0.0705

Epoch 00001: val_loss improved from inf to 0.07053, saving model to gdrive/My Drive/Colab Notebooks/tripletloss/weights.hdf5
Epoch 2/10
900/900 [==============================] - 9983s 11s/step - loss: 0.0118 - val_loss: 0.0263

Epoch 00002: val_loss improved from 0.07053 to 0.02625, saving model to gdrive/My Drive/Colab Notebooks/tripletloss/weights.hdf5
Epoch 3/10
900/900 [==============================] - 9993s 11s/step - loss: 9.1996e-04 - val_loss: 0.0369

Epoch 00003: val_loss did not improve from 0.02625
Epoch 4/10
 10/900 [..............................] - ETA: 2:39:19 - loss: 0.0000e+00

In [0]:
trained_model = Model(inputs=anchor_input, outputs=encoded_anchor)

In [15]:
trained_model.load_weights(root_path + "weights.hdf5")

In [0]:
X_train_trm = trained_model.predict(dataset.images_train.reshape(-1,480, 640, 1))

In [17]:
len(X_train_trm)

922

In [0]:
X_train_trm_new = X_train_trm[:850]
X_test_trm = X_train_trm[850:922]

In [20]:
y_train_onehot = np.zeros((dataset.labels_train.size, dataset.labels_train.max()+1))
y_train_onehot[np.arange(dataset.labels_train.size),dataset.labels_train] = 1

print(len(y_train_onehot))
y_train_onehot_new = y_train_onehot[:850]
y_test_onehot = y_train_onehot[850:922]

dataset.labels_test
# y_test_onehot = np.zeros((dataset.labels_test.size, dataset.labels_test.max()+1))
# y_test_onehot[np.arange(dataset.labels_test.size),dataset.labels_test] = 1
# dataset.labels_test.max()
print(len(y_test_onehot))

922
72


In [0]:
Classifier_input = Input((4,))
Classifier_output = Dense(28, activation='softmax')(Classifier_input)
Classifier_model = Model(Classifier_input, Classifier_output)

Classifier_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [25]:
Classifier_model.fit(X_train_trm_new,y_train_onehot_new, validation_data=(X_test_trm,y_test_onehot),epochs=200)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 850 samples, validate on 72 samples
Epoch 1/200
850/850 [==============================] - 0s 459us/step - loss: 3.7452 - acc: 0.0000e+00 - val_loss: 3.8151 - val_acc: 0.0000e+00
Epoch 2/200
850/850 [==============================] - 0s 38us/step - loss: 3.5610 - acc: 0.0000e+00 - val_loss: 3.6400 - val_acc: 0.0000e+00
Epoch 3/200
850/850 [==============================] - 0s 39us/step - loss: 3.3961 - acc: 0.0094 - val_loss: 3.4814 - val_acc: 0.0556
Epoch 4/200
850/850 [==============================] - 0s 38us/step - loss: 3.2471 - acc: 0.0600 - val_loss: 3.3378 - val_acc: 0.1111
Epoch 5/200
850/850 [==============================] - 0s 37us/step - loss: 3.1140 - acc: 0.1259 - val_loss: 3.2018 - val_acc: 0.1667
Epoch 6/200
850/850 [==============================] - 0s 38us/step - loss: 2.9935 - acc: 0.1941 - val_loss: 3.0800 - val_acc: 0.2083
Epoch 7/200
850/850 [=================